# 04. Test MLflow Serving (Docker)

Ce notebook teste l'API exposée par le conteneur Docker.
Assurez-vous que le conteneur tourne sur le port 5000.
Commande: `docker run -p 5000:5000 -v $(pwd)/models/final_model:/model -e MLFLOW_MODEL_URI=/model credit-scoring-serving`

In [26]:
import requests
import pandas as pd
import json
import numpy as np

In [27]:
# Chargement d'un sample de données (V1 ou V2, peu importe tant que le format colonnes est bon)
# Idéalement prendre le même dataset que celui du gagnant
X_test = pd.read_pickle('../data/processed/X_prepared_v2.pkl')
y_test = pd.read_pickle('../data/processed/y_prepared_v2.pkl')
sample = X_test.head(5)
resultat = y_test.head(5)
print("Sample shape:", sample.shape)
print("Resultat shape:", resultat.shape)


Sample shape: (5, 124)
Resultat shape: (5,)


In [28]:
# Endpoint par defaut MLflow serve
url = "http://127.0.0.1:5000/invocations"

# Copier le sample pour ne pas modifier le DataFrame original
sample_cleaned = sample.copy()

# Remplacer les valeurs non JSON (NaN, inf) par 0.0 pour l'envoi.
sample_cleaned.replace([np.nan, np.inf, -np.inf], 0.0, inplace=True) 

# on utilise le sample nettoyé
data_json = sample_cleaned.to_dict(orient='split') 

headers = {'Content-Type': 'application/json'}

try:
    response = requests.post(url, json={"dataframe_split": data_json}, headers=headers)
    print("Status Code:", response.status_code)
    
    if response.status_code == 200:
        print("Predictions:", response.json())
        print("Vrai résultat\n", resultat.values)
    else:
        print("Erreur:", response.text)
except Exception as e:
    print("Erreur connexion:", e)
    print("Vérifiez que le docker tourne bien.")

Status Code: 200
Predictions: {'predictions': [1, 0, 0, 0, 1]}
Vrai résultat
 [1 0 0 0 0]
